<a href="https://colab.research.google.com/github/mariasakorafou/maria1/blob/main/IPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
#εγκατάσταση πακέτου tensorflow (hhttps://www.tensorflow.org/)
!pip install tensorflow
#εγκατάσταση πακέτου scikit-learn (https://scikit-learn.org/)
!pip install scikit-learn





In [45]:
#εισαγωγή βιβλιοθηκών - πακέτων
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import smtplib
from email.mime.text import MIMEText



In [46]:
# Συνάρτηση αποστολής ειδοποίησης διαχειριστή
def notify_admin(threat_details):
    sender_email = "sakorafou.maria@gmail.com"
    sender_password = "George43!maria"
    recipient_email = "msakorafou@yahoo.gr"
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    message = MIMEText(f"Threat detected:\n{threat_details}")
    message["Subject"] = "Intrusion Detected"
    message["From"] = sender_email
    message["To"] = recipient_email

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, message.as_string())
        print("Notification sent to admin.")
    except Exception as e:
        print(f"Failed to send notification: {e}")

In [47]:
#συνάρτηση εισαγωγής συνόλου δεδομένων
url = "https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/pcap_data.csv"
data = pd.read_csv(url)


In [48]:
#Προεπεξεργασία δεδομένων
#Ελέγχουμε για κενές τιμές και τις αντικαθιστούμε ή αφαιρούμε
data.dropna(inplace=True)

In [49]:
#select features
X = data.drop(columns=['target'])  #  στήλη στόχος
y = data['target']


In [50]:
#normalization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)



In [51]:
#training
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
# Βήμα 5: Αξιολόγηση Μοντέλου
y_pred = model.predict(X_test)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Προληπτικές ενέργειες βάσει εντοπισμένων απειλών
for i, (features, prediction) in enumerate(zip(X_test, y_pred)):
    if prediction != 0:  # 0 σημαίνει "κανονική κυκλοφορία"
        threat_details = f"Threat detected in sample {i}: Class {prediction}"
        print(threat_details)

        # Ενέργειες IPS
        if prediction == 1:
            print("Action: Blocking suspicious traffic...")
            # Εισαγωγή κανόνων στο firewall
        elif prediction == 2:
            print("Action: Isolating affected devices...")
            # Εφαρμογή πολιτικών δικτύου
        notify_admin(threat_details)

Confusion Matrix:
[[1341   81   76]
 [  75 1389   55]
 [  67   55 1352]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.90      0.90      0.90      1498
         1.0       0.91      0.91      0.91      1519
         2.0       0.91      0.92      0.91      1474

    accuracy                           0.91      4491
   macro avg       0.91      0.91      0.91      4491
weighted avg       0.91      0.91      0.91      4491

Threat detected in sample 2: Class 2.0
Action: Isolating affected devices...
Failed to send notification: Connection unexpectedly closed
Threat detected in sample 5: Class 2.0
Action: Isolating affected devices...
Failed to send notification: Connection unexpectedly closed
Threat detected in sample 6: Class 2.0
Action: Isolating affected devices...
Failed to send notification: Connection unexpectedly closed
Threat detected in sample 8: Class 2.0
Action: Isolating affected devices...
Failed to send notification: Conne